# Defaults & Charge-off Prediction

## Data Preprocessing

In [46]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
# load the data
loan3a = pd.read_csv('../data/LoanStats3a_securev1.csv', header=1, low_memory=False)
loan3b = pd.read_csv('../data/LoanStats3b_securev1.csv', header=1, low_memory=False)
loan3c = pd.read_csv('../data/LoanStats3c_securev1.csv', header=1, low_memory=False)
loan3d = pd.read_csv('../data/LoanStats3d_securev1.csv', header=1, low_memory=False)

In [48]:
loan3a

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42533,72176,70868.0,2525.0,2525.0,225.0,36 months,9.33%,80.69,B,B3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42534,71623,70735.0,6500.0,6500.0,0.0,36 months,8.38%,204.84,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42535,70686,70681.0,5000.0,5000.0,0.0,36 months,7.75%,156.11,A,A3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42536,Total amount funded in policy code 1: 471701350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
# combine the data
df = pd.concat([loan3a, loan3b, loan3c, loan3d], axis=0)

In [50]:
df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421092,36271333,38982739.0,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,D2,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806,39222577.0,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,E3,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262,38982659.0,20000.0,20000.0,20000.0,36 months,11.99%,664.20,B,B5,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# remove first dataframes from memory
del loan3a
del loan3b
del loan3c
del loan3d

In [52]:
# display all types of loan_status
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', 'Current', 'Late (31-120 days)',
       'In Grace Period', 'Late (16-30 days)', 'Default', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

### Drop Unnecessary Variables

In [53]:
# drop the columns that are not needed
df.drop(['id', 'member_id', 'emp_title', 'pymnt_plan', 'url', 'desc', 'title', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint'], axis=1, inplace=True)

# drop columns with more than 90% missing values
missing_values = df.isnull().mean()
missing_values = missing_values[missing_values > 0.9]

df.drop(missing_values.index, axis=1, inplace=True)

In [54]:
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,10+ years,RENT,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,< 1 year,RENT,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,10+ years,RENT,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,10+ years,RENT,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,1 year,RENT,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421092,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,D2,5 years,RENT,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,E3,1 year,RENT,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,20000.0,20000.0,20000.0,36 months,11.99%,664.20,B,B5,10+ years,RENT,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Handle Missing Values

In [55]:
# for any column starting with 'mths_since', fill missing values with -1
mths_since_columns = [col for col in df.columns if col.startswith('mths_since') or col.startswith('mo_sin')]
df[mths_since_columns] = df[mths_since_columns].fillna(-1)

In [56]:
# for any nan values in next_pymnt_d, fill with 'No Payment Plan'
df['next_pymnt_d'] = df['next_pymnt_d'].fillna('No Payment Plan')

In [57]:
# drop rows with missing values
df.dropna(inplace=True)

In [58]:
df

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,3000.0,3000.0,3000.0,36 months,12.85%,100.87,B,B4,10+ years,RENT,...,0.0,1.0,91.3,66.7,2.0,0.0,32082.0,19530.0,4300.0,26782.0
1,7550.0,7550.0,7550.0,36 months,16.24%,266.34,C,C5,3 years,RENT,...,0.0,0.0,100.0,100.0,0.0,0.0,8000.0,5759.0,4000.0,0.0
2,20800.0,20800.0,20800.0,36 months,13.53%,706.16,B,B5,10+ years,RENT,...,0.0,3.0,90.2,50.0,0.0,0.0,43100.0,23473.0,15000.0,0.0
3,15000.0,15000.0,15000.0,36 months,8.90%,476.30,A,A5,2 years,MORTGAGE,...,0.0,0.0,89.3,66.7,0.0,0.0,288195.0,39448.0,14200.0,33895.0
4,9750.0,9750.0,9750.0,36 months,13.98%,333.14,C,C1,1 year,RENT,...,0.0,2.0,100.0,66.7,0.0,0.0,21314.0,14123.0,7200.0,6214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421090,10000.0,10000.0,10000.0,36 months,11.99%,332.10,B,B5,8 years,RENT,...,0.0,1.0,100.0,100.0,0.0,0.0,32950.0,25274.0,9200.0,15850.0
421091,24000.0,24000.0,24000.0,36 months,11.99%,797.03,B,B5,10+ years,MORTGAGE,...,0.0,2.0,56.5,100.0,0.0,0.0,152650.0,8621.0,9000.0,0.0
421092,13000.0,13000.0,13000.0,60 months,15.99%,316.07,D,D2,5 years,RENT,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,12000.0,12000.0,12000.0,60 months,19.99%,317.86,E,E3,1 year,RENT,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0


## Feature Engineering

In [59]:
# print data types of all columns
df.dtypes

loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 89, dtype: object

In [60]:
# drop zip code and state columns
df.drop(['zip_code', 'addr_state'], axis=1, inplace=True)

In [61]:
# convert int_rate from percentage to float
df['int_rate'] = df['int_rate'].str.replace('%', '').astype(float)

# convert revol_util from percentage to float
df['revol_util'] = df['revol_util'].str.replace('%', '').astype(float)

In [62]:
# one hot encoding for categorical variables
df = pd.get_dummies(df, drop_first=True)

In [64]:
# convert one hot encoded columns to 1's and 0's
df = df.astype(int)

In [65]:
df

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,...,last_credit_pull_d_Nov-2014,last_credit_pull_d_Nov-2015,last_credit_pull_d_Oct-2012,last_credit_pull_d_Oct-2013,last_credit_pull_d_Oct-2014,last_credit_pull_d_Oct-2015,last_credit_pull_d_Sep-2012,last_credit_pull_d_Sep-2013,last_credit_pull_d_Sep-2014,last_credit_pull_d_Sep-2015
0,3000,3000,3000,12,100,25000,24,0,660,664,...,0,0,0,0,0,0,0,0,0,0
1,7550,7550,7550,16,266,28000,8,0,660,664,...,0,0,0,0,0,0,0,0,0,0
2,20800,20800,20800,13,706,81500,16,0,685,689,...,0,0,0,0,0,0,0,0,0,0
3,15000,15000,15000,8,476,63000,16,0,670,674,...,0,0,0,0,0,0,0,0,0,0
4,9750,9750,9750,13,333,26000,25,0,670,674,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421090,10000,10000,10000,11,332,31000,28,0,670,674,...,0,0,0,0,0,0,0,0,0,0
421091,24000,24000,24000,11,797,79000,3,0,660,664,...,0,0,0,0,0,0,0,0,0,0
421092,13000,13000,13000,15,316,35000,30,0,680,684,...,0,0,0,0,0,0,0,0,0,0
421093,12000,12000,12000,19,317,64400,27,1,695,699,...,0,0,0,0,0,0,0,0,0,0
